**CASO:**

Uno de nuestros clientes, Atlas Bank, que se dedica al sector de finanzas quiere conocer
mejor la satisfacción de sus clientes.
Actualmente tienen una encuesta en su página web que les permite recopilar niveles de
satisfacción de 1 a 5 y comentarios.

In [ ]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Lee un archivo de valores separados por comas (csv) en un DataFrame
df = pd.read_csv('data.csv')

**Exploratory Data Analysis (EDA)**: Análisis Exploratorio de Datos

In [ ]:
# Devuelve una tupla que representa la dimensionalidad del DataFrame
df.shape

(11050, 6)

In [ ]:
# Esta función devuelve las primeras 3 filas del objeto según la posición
df.head(3)

,Fecha,Pais,Dispositivo,Browser,Rank,Comentario
0,8/8/2019,Peru,phone,Chrome Mobile 76.0.3809,1,No consigo lo que deseo. No consigo agendar un...
1,8/8/2019,Peru,desktop,Chrome 76.0.3809,1,NaN
2,8/9/2019,Peru,phone,Chrome Mobile 42.0.2311,1,NaN


In [ ]:
# Revisa el tipo de datos en las columnas
df.dtypes

Fecha          object
Pais           object
Dispositivo    object
Browser        object
Rank            int64
Comentario     object
dtype: object

In [ ]:
# Revisa si existen valores nulos
df.isna().sum()

Fecha             0
Pais              0
Dispositivo       0
Browser           0
Rank              0
Comentario     7540
dtype: int64

**1. Cuantificar el número de usuarios felices/ infelices**

In [ ]:
# Suma la cantidad de valoraciones '4' y '5' de la columna rank
q_u_felices = int(df['Rank'].value_counts().loc[4])+int(df['Rank'].value_counts().loc[5])
# Suma la cantidad de valoraciones '1', '2' y '3' de la columna rank
q_u_infelices = int(df['Rank'].value_counts().loc[1])+int(df['Rank'].value_counts().loc[2])+int(df['Rank'].value_counts().loc[3])

# Crea un dataframe para visualiación
tabla = pd.DataFrame(
    {
        'Cantidad': [q_u_felices, q_u_infelices],
    },
    index=[
           'Usuarios felices', 'Usuarios infelices'
    ])
tabla.reset_index(inplace=True)
tabla

,index,Cantidad
0,Usuarios felices,4876
1,Usuarios infelices,6174


**2. Listar sentimientos negativos/ positivos**

In [ ]:
# Instala la biblioteca pysentimiento que utiliza modelos previamente entrenados de transformers para diferentes tareas de NLP Social.
# Utiliza como modelos base a [BETO] (https://github.com/dccuchile/beto) y [RoBERTuito] (https://github.com/pysentiment/robertuito) en español
!pip install pysentimiento

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# Elimina las filas en las cuales no existen comentarios
df_sentimiento = df[df['Comentario'].notna()]
df_sentimiento.shape

(3510, 6)

In [ ]:
df_sentimiento.head(3)

,Fecha,Pais,Dispositivo,Browser,Rank,Comentario
0,8/8/2019,Peru,phone,Chrome Mobile 76.0.3809,1,No consigo lo que deseo. No consigo agendar un...
4,8/9/2019,Peru,phone,Chrome Mobile 75.0.3770,1,No se como registrarme para mi clave
6,8/10/2019,Peru,phone,Mobile Safari 12.1.2,1,Sale otro idioma 😡


In [ ]:
def remover_emojis(inputString):
    """
    Remueve emojis de un texto y los reemplza con un string vacío
    """

    import re

    # Expresión regular que coincide con una lista de emojis
    emoji_pattern = re.compile("["
    u"\U0001F600-\U0001F64F" # emoticons
    u"\U0001F300-\U0001F5FF" # symbols & pictographs
    u"\U0001F680-\U0001F6FF" # transport & map symbols
    u"\U0001F1E0-\U0001F1FF" # flags (iOS)
    u"\U00002702-\U000027B0"
    u"\U000024C2-\U0001F251"
    u"\U0001f926-\U0001f937"
    u'\U00010000-\U0010ffff'
    u"\u200d"
    u"\u2640-\u2642"
    u"\u2600-\u2B55"
    u"\u23cf"
    u"\u23e9"
    u"\u231a"
    u"\u3030"
    u"\ufe0f"
    u"\u2069"
    u"\u2066"
    u"\u200c"
    u"\u2068"
    u"\u2067"
            "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', inputString)

In [ ]:
# Aplica la función para remover emojis puesto que pysentimiento aún no funciona eficazmente con emojis
df_sentimiento['Comentario'] = df_sentimiento['Comentario'].apply(remover_emojis)
df_sentimiento.head(3)

,Fecha,Pais,Dispositivo,Browser,Rank,Comentario
0,8/8/2019,Peru,phone,Chrome Mobile 76.0.3809,1,No consigo lo que deseo. No consigo agendar un...
4,8/9/2019,Peru,phone,Chrome Mobile 75.0.3770,1,No se como registrarme para mi clave
6,8/10/2019,Peru,phone,Mobile Safari 12.1.2,1,Sale otro idioma


In [ ]:
# Coloca NaN a los comentarios que quedan vacíos
df_sentimiento = df_sentimiento.replace(r'^\s*$', np.nan, regex=True)
# Elimina los comentarios con valores NaN
df_sentimiento_limpio = df_sentimiento[df_sentimiento['Comentario'].notna()]
df_sentimiento_limpio.shape

(3480, 6)

In [ ]:
def analizador_sentimiento(df):
    """
    Analiza
    """

    from pysentimiento import create_analyzer
    # Crea un analizador
    analizador = create_analyzer(task="sentiment", lang="es")

    analisis = []

    from tqdm.auto import tqdm

    for ind in tqdm(df.index):
        analisis.append(str(analizador.predict(df['Comentario'][ind])).split('=')[1][:3])

    df['Análisis_Sentimiento'] = analisis

    df['Análisis_Sentimiento'] = np.where((df['Análisis_Sentimiento'] == 'NEU'),'Neutro',df['Análisis_Sentimiento'])
    df['Análisis_Sentimiento'] = np.where((df['Análisis_Sentimiento'] == 'NEG'),'Negativo',df['Análisis_Sentimiento'])
    df['Análisis_Sentimiento'] = np.where((df['Análisis_Sentimiento'] == 'POS'),'Positivo',df['Análisis_Sentimiento'])

    return df

In [ ]:
%%time
# Aplica la función para analizar sentimiento sobre el dataframe limpio
analizador_sentimiento(df_sentimiento_limpio)
df_sentimiento_limpio.head()

  0%|          | 0/3480 [00:00<?, ?it/s]

CPU times: user 7min 6s, sys: 6.54 s, total: 7min 13s
Wall time: 7min 28s


,Fecha,Pais,Dispositivo,Browser,Rank,Comentario,Análisis_Sentimiento
0,8/8/2019,Peru,phone,Chrome Mobile 76.0.3809,1,No consigo lo que deseo. No consigo agendar un...,Negativo
4,8/9/2019,Peru,phone,Chrome Mobile 75.0.3770,1,No se como registrarme para mi clave,Neutro
6,8/10/2019,Peru,phone,Mobile Safari 12.1.2,1,Sale otro idioma,Neutro
10,8/10/2019,Peru,desktop,Chrome 76.0.3809,1,NO PUEDO BLOQUEAR MI TARJETA MEDIANTE WEB :l,Negativo
11,8/10/2019,Peru,phone,Chrome Mobile 76.0.3809,1,No encuentro informacion como validar mi nueva...,Neutro


In [ ]:
# Calcula la cantidad de sentimientos positivos, negativos y neutros
tabla_sentimiento = df_sentimiento_limpio['Análisis_Sentimiento'].value_counts().rename_axis('Tipo de Comentario').to_frame('Cantidad')
tabla_sentimiento.reset_index(inplace=True)
tabla_sentimiento

,Tipo de Comentario,Cantidad
0,Negativo,1644
1,Neutro,1272
2,Positivo,564


**3. Identificar y priorizar oportunidades para mejorar de la experiencia del consumidor mediante los comentarios de los usuarios digitales**

In [ ]:
!pip install stop_words
!pip install stylecloud

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from stop_words import get_stop_words

palabras_irrelevantes = get_stop_words('spanish')
palabras_irrelevantes[:5]

['a', 'al', 'algo', 'algunas', 'algunos']

In [ ]:
import stylecloud

In [ ]:
df_comentarios_positivos = df_sentimiento_limpio.loc[df_sentimiento_limpio['Análisis_Sentimiento'] == 'Positivo'].copy()
comentarios_positivos = ' '.join([comentario for comentario in df_comentarios_positivos['Comentario']])
comentarios_positivos

'Interbank es un buen banco No hay como hablar con soporte BBVA es el mejor bancoooooooo muchoo mejr es bbva Muy alto interés Continental es mejor De muy buena onda banco azteca tiene mejores tasas Bien Bien Muy bien  Bien  this is good execelente Bien  Bien Bien Me parecio concreto e interactivo Todo bien por ahora me gusta es como si te atendieran personalmente Excelente. Fácil de usar. Esta muy bien asy\n Bien Si me gusta Muy bueno Me gusta mucho Buena pagina Muy bueno.. Bieen me gusta ! Bn Bien Excelente oportunidad para adquirir   un \n Muy bien Excelente servicio Espetacular Me gusta el fácil y rápido  Exelente\n Me gusta mucho Bien\n Bien Muy bien Excelente  Excelente  Bien  Muy interesante\n Muy bueno Muy bien  Exelente las promociones son buenas Es buena esta experiencia que lo disfruto con ansias de cumplir mi sueño de la casa propia  Muy buena Bien bien Bien  Muy buena información  Muy bueno AAA Muy buena y rápida Muy bueno Excelente Bien  Bien  muy bien Bien buena  experien

In [ ]:
stylecloud.gen_stylecloud(comentarios_positivos, icon_name='fas fa-book-open', custom_stopwords=palabras_irrelevantes, output_name='nube_comentarios_positivos.png')

In [ ]:
df_comentarios_neutros = df_sentimiento_limpio.loc[df_sentimiento_limpio['Análisis_Sentimiento'] == 'Neutro'].copy()
comentarios_neutros = ' '.join([comentario for comentario in df_comentarios_neutros['Comentario']])
comentarios_neutros

'No se como registrarme para mi clave Sale otro idioma  No encuentro informacion como validar mi nueva tarjeta de debito a fin de poder registrarla para ingresar No encuentro un número al que pueda llamar desde los EEUU. El número que tienen publicado es raro. Ni siquiera es un número normal del Perú. Y para chatear tengo que inscribirme con algún ID. Qué ID? No tengo ID peruano. no sep uede realizar pagos\nindica donde Aún no puedo entrar a préstamo vía wed  Y No pasa nada . como acceder por internet necesito numero de cuennta Quiero hacer consultas Es imposible  Mucho q llenar No respondes  Hola ViaAtlas Bank\n M ayudaaaaaaaaa Feo Estoy buscando donde ver mis movimientos .Cuento con la clave de 4 dígitos No pasa nada cuando presiono enviar No acepta\n qu{e objetivo tiene el captcha<? tengo problemas en la vista, por lo que me equivoco digitar la tarjeta y el captcha, podrian facilitar poniendo la opcion de lupa para no demorar e ingrese en una sola vez y no en varias, o de lo contrar

In [ ]:
stylecloud.gen_stylecloud(comentarios_neutros, icon_name='fas fa-book-open', custom_stopwords=palabras_irrelevantes, output_name='nube_comentarios_neutros.png')

In [ ]:
df_comentarios_negativos = df_sentimiento_limpio.loc[df_sentimiento_limpio['Análisis_Sentimiento'] == 'Negativo'].copy()
comentarios_negativos = ' '.join([comentario for comentario in df_comentarios_negativos['Comentario']])
comentarios_negativos

'No consigo lo que deseo. No consigo agendar una llamada NO PUEDO BLOQUEAR MI TARJETA MEDIANTE WEB :l Ahora piden la clave token que no llega ni al correo, ni al celuar. Arreglen eso Quiero henerar clave y no puedo\n No me atienden  no entiendo nada !\n nps sunat nunca funciona\n Pesima No puedo abrir el aPp No sale como crear mi clave x internet es una perdida de tiempo\n No hay nada y no veo opcion para obtener mi clave por internet Malaasss Quiero optener mi clave por internet.. y no puedo No carga la web clave de internet recupere mi tarjeta en la agencia y la srta no me ayudó para configurar la app del Atlas Bank  es dificil entrar banca por intenet Malo muy malo Quiero mi clave y no puedo es una tontera, no se directo al grano No puedo obtener mi clsve No encuentro con crear mi clave de 4 dígitos  Una mrd no se entiende un carajo Me molesta q no hay sistema no puedo entrar a banca por internet\n No se cómo crear mi clave de 4 dígitos  Primero en banca por internet me pidio la con

In [ ]:
stylecloud.gen_stylecloud(comentarios_negativos, icon_name='fas fa-book-open', custom_stopwords=palabras_irrelevantes, output_name='nube_comentarios_negativos.png')

In [ ]:
from collections import Counter

In [ ]:
# Remove substring list from String
# Using loop + replace()
palabras_irrelevantes.extend(['No', 'puedo', 'banca'])
for sub in palabras_irrelevantes:
    comentarios_negativos = comentarios_negativos.replace(' ' + sub + ' ', ' ')

In [ ]:
comentarios_negativos

'No consigo deseo. consigo agendar llamada NO PUEDO BLOQUEAR MI TARJETA MEDIANTE WEB :l Ahora piden clave token llega correo, celuar. Arreglen Quiero henerar clave puedo\n atienden  entiendo !\n nps sunat nunca funciona\n Pesima abrir aPp sale crear clave x internet perdida tiempo\n veo opcion obtener clave internet Malaasss Quiero optener clave internet.. carga web clave internet recupere tarjeta agencia srta ayudó configurar app Atlas Bank  dificil entrar intenet Malo malo Quiero clave tontera, directo grano obtener clsve encuentro crear clave 4 dígitos  Una mrd entiende carajo Me molesta q sistema entrar internet\n cómo crear clave 4 dígitos  Primero internet pidio contraseña cajero raro h luego salio fallido pidio contraseña internet parecio raro accesar cuentas hacer transferencias disponible cta. ... Mortifica mucho. Mala contestaron llamada quiero cancelar póliza tarjeta débito La atención ineficiente\n dice días atención teléfono fin semana imposible atienden. La verdad perdido

In [ ]:
# split() returns list of all the words in the string
split_it = comentarios_negativos.split()

# Pass the split_it list to instance of Counter class.
Counter = Counter(split_it)

# most_common() produces k frequently encountered
# input values and their respective counts.
most_occur = Counter.most_common(4)

print(most_occur)

[('clave', 188), ('internet', 158), ('cuenta', 128), ('ingresar', 108)]


In [ ]:
contain_values = df_comentarios_negativos.loc[df_comentarios_negativos['Comentario'].str.contains('clave')]
muestra = contain_values.sample(5)['Comentario'].to_frame()
muestra

,Comentario
9611,"Aun no entiendo como obtener la clave OTP, aqu..."
6789,no se puede generar la clave
7225,no me permite crear mi clave de internet
9320,Hola xq no puedo generar mi clave para utiliza...
6791,Porque me rechasa si mi clave está bien puesta...


**4. Insight adicional**

In [ ]:
df['Pais'].value_counts()[:5]

Peru             10694
United States      110
Spain               54
Chile               34
Bolivia             22
Name: Pais, dtype: int64

In [ ]:
df['Dispositivo'].value_counts()

phone      8438
desktop    2522
tablet       90
Name: Dispositivo, dtype: int64

In [ ]:
df['Browser'].value_counts()[:5]

Chrome Mobile 76.0.3809    1214
Chrome Mobile 77.0.3865    1144
Chrome Mobile 79.0.3945    1016
Chrome Mobile 78.0.3904     878
Chrome 79.0.3945            670
Name: Browser, dtype: int64

In [ ]:
def formateador_fecha(df):
    """
    Transforma la fecha
    """

    Fecha_formateada = []

    for ind in df.index:
        if "/" in df['Fecha'][ind]:
            Fecha_formateada.append(f"{df['Fecha'][ind][-4:]}-{df['Fecha'][ind].split('/')[0]}-{df['Fecha'][ind].split('/')[1]} 00:00:00")
        else:
            Fecha_formateada.append(df['Fecha'][ind])

    df['Fecha_formateada'] = Fecha_formateada

    # convert the 'Date' column to datetime format
    df['Fecha_formateada']= pd.to_datetime(df['Fecha_formateada'])

    df = df.drop(columns=['Fecha'], axis=1, inplace=True)

    return df

In [ ]:
formateador_fecha(df)
df.head(3)

,Pais,Dispositivo,Browser,Rank,Comentario,Fecha_formateada
0,Peru,phone,Chrome Mobile 76.0.3809,1,No consigo lo que deseo. No consigo agendar un...,2019-08-08
1,Peru,desktop,Chrome 76.0.3809,1,NaN,2019-08-08
2,Peru,phone,Chrome Mobile 42.0.2311,1,NaN,2019-08-09


In [ ]:
df['Fecha_formateada'] = df['Fecha_formateada'].dt.to_period('M')
df.head(3)

,Pais,Dispositivo,Browser,Rank,Comentario,Fecha_formateada
0,Peru,phone,Chrome Mobile 76.0.3809,1,No consigo lo que deseo. No consigo agendar un...,2019-08
1,Peru,desktop,Chrome 76.0.3809,1,NaN,2019-08
2,Peru,phone,Chrome Mobile 42.0.2311,1,NaN,2019-08


**Dashboard**

In [ ]:
!pip install jupyter-dash
!pip install dash-bootstrap-components

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import plotly.graph_objects as go
import plotly.express as px
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
import dash_table

**Para que Dash cargue las imágene**s, se debe de crear una carpeta "assets" y mover las imagenes dentro de ella.

In [ ]:
# Contruye las tablas
table = dash_table.DataTable(tabla.to_dict('records'), [{"name": i, "id": i} for i in tabla.columns])
table_2 = dash_table.DataTable(tabla_sentimiento.to_dict('records'), [{"name": i, "id": i} for i in tabla_sentimiento.columns])
table_3 = dash_table.DataTable(muestra.to_dict('records'), [{"name": i, "id": i} for i in muestra.columns])

# Construye la app jupyter dash
app = JupyterDash(__name__)

# Agrega los elementos HTML
app.layout = html.Div(
    [
      html.H1('DASHBOARD ATLAS BANK', style={'color': 'red'}),
      html.Br(),
      html.H2('Número de usuarios felices e infelices', style={'color': 'red'}),
      table,
      html.Br(),
      html.H2('Análisis de Sentimientos', style={'color': 'red'}),
      table_2,
      html.Br(),
      html.H2('Oportunidades para Mejorar:', style={'color': 'red'}),
      html.H3('Comentarios positivos', style={'color': 'red'}),
      html.Img(src=app.get_asset_url('nube_comentarios_positivos.png')),
      html.H3('Comentarios neutros', style={'color': 'red'}),
      html.Img(src=app.get_asset_url('nube_comentarios_neutros.png')),
      html.H3('Comentarios negativos', style={'color': 'red'}),
      html.Img(src=app.get_asset_url('nube_comentarios_negativos.png')),
      table_3
    ]
  )

# Corre el dashboard en el navegador
app.run_server(mode='external') #"inline" corre el dashboard in Colab, "external" corre el Dashboard en el navegador via localhost

Dash app running on:


<IPython.core.display.Javascript object>